<a href="https://colab.research.google.com/github/Richish/hands_on_ml/blob/master/7_3_ensemble_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the MNIST data, and split it into a training set, a
validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation,and 10,000 for testing). Then train various classifiers, such as a Random Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft or hard voting classifier. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [3]:
mnist.data.shape, mnist.target.shape, mnist.target_names #,mnist.feature_names, 

((70000, 784), (70000,), ['class'])

In [4]:
X_train, y_train, X_val, y_val, X_test, y_test  = mnist.data[:50000], mnist.target[:50000], mnist.data[50000:60000], mnist.target[50000:60000], mnist.data[60000:], mnist.target[60000:]
X_train.shape, y_test.shape

((50000, 784), (10000,))

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=10, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf_clf = rf_clf.predict(X_val)
accuracy_score(y_true=y_val, y_pred=y_pred_rf_clf)

0.7903

In [6]:
from sklearn.ensemble import ExtraTreesClassifier

xt_clf = ExtraTreesClassifier(n_estimators=100, max_leaf_nodes=10, random_state=42)
xt_clf.fit(X_train, y_train)
y_pred_xt_clf = xt_clf.predict(X_val)
accuracy_score(y_true=y_val, y_pred=y_pred_xt_clf)

0.7762

In [7]:
"""
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

svc_clf = Pipeline([
                               ("scaler", StandardScaler()),
                               ("svm_clf", LinearSVC(C=1.0, loss="hinge"))
])
svc_clf.fit(X_train, y_train)
y_pred_svc_clf = svc_clf.predict(X_val)

#svc_clf = SVC(probability=True)
#svc_clf.fit(X_train, y_train)
#y_pred_svc_clf = svc_clf.predict(X_val)
accuracy_score(y_true=y_val, y_pred=y_pred_svc_clf)
"""

'\nfrom sklearn.svm import LinearSVC\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.pipeline import Pipeline\n\nsvc_clf = Pipeline([\n                               ("scaler", StandardScaler()),\n                               ("svm_clf", LinearSVC(C=1.0, loss="hinge"))\n])\nsvc_clf.fit(X_train, y_train)\ny_pred_svc_clf = svc_clf.predict(X_val)\n\n#svc_clf = SVC(probability=True)\n#svc_clf.fit(X_train, y_train)\n#y_pred_svc_clf = svc_clf.predict(X_val)\naccuracy_score(y_true=y_val, y_pred=y_pred_svc_clf)\n'

In [8]:
from sklearn.ensemble import VotingClassifier

hv_clf = VotingClassifier(estimators=[('rf',rf_clf),('xt',xt_clf)], voting='hard' )
hv_clf.fit(X_train, y_train)
hv_clf_pred = hv_clf.predict(X_val)
accuracy_score(y_true=y_val, y_pred=hv_clf_pred)

0.7708

In [9]:
sv_clf = VotingClassifier(estimators=[('rf',rf_clf),('xt',xt_clf)], voting='soft' )
sv_clf.fit(X_train, y_train)
sv_clf_pred = sv_clf.predict(X_val)
accuracy_score(y_true=y_val, y_pred=sv_clf_pred)

0.7935

In [10]:
accuracy_score(y_true=y_test, y_pred=rf_clf.predict(X_test)), accuracy_score(y_true=y_test, y_pred=xt_clf.predict(X_test)), accuracy_score(y_true=y_test, y_pred=sv_clf.predict(X_test))




(0.7757, 0.7635, 0.7804)

Run the individual classifiers from the previous exercise to make predictions on
the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image’s class. Train a classifier on this new training set. You have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let’s evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble’s predictions.

How does it compare to the voting classifier you trained earlier?

In [11]:
y_pred_rf_clf.shape, y_pred_xt_clf.shape


((10000,), (10000,))

In [23]:
import numpy as np
X_train_new = np.concatenate((y_pred_rf_clf.reshape(-1,1), y_pred_xt_clf.reshape(-1,1)), axis=1)
X_train_new.shape

(10000, 2)

In [31]:
# using xgboost as blender
import xgboost

xgb_clf = xgboost.XGBClassifier()
xgb_clf.fit(X_train_new, y_val)
X_test_new = np.concatenate((rf_clf.predict(X_test).reshape(-1,1),xt_clf.predict(X_test).reshape(-1,1)), axis=1)
y_pred = xgb_clf.predict(X_test_new)
y_pred.shape, y_test.shape
accuracy_score(y_true=y_test, y_pred=y_pred)

0.7891